In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [2]:
print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('Scikit-Learn : %s'%(sklearn.__version__))
!python --version


Pandas : 1.0.1
Numpy : 1.18.1
Scikit-Learn : 0.22.1
Python 3.7.6


In [3]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [5]:
# 날짜 처리
data = pd.read_csv('201901-202003.csv')
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [6]:
# 데이터 정제
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [7]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [10]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [11]:
# 훈련
model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(train_features, train_target)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

# 여기서 부터 예측테이블 -> submission까지 과정

In [90]:
predict_csv = pd.read_csv('predict_table.csv',encoding='UTF-8')
predict_csv = predict_csv.drop('Unnamed: 0',axis=1)
predict_csv.head()


,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,강원,건강보조식품 소매업,강원,20s,2,1,2020,4
1,강원,건강보조식품 소매업,강원,20s,2,1,2020,7
2,강원,건강보조식품 소매업,강원,20s,2,2,2020,4
3,강원,건강보조식품 소매업,강원,20s,2,2,2020,7
4,강원,건강보조식품 소매업,강원,30s,1,2,2020,4


In [83]:
predict_num = predict_csv.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    predict_num[column] = encoder.transform(predict_num[column])


In [84]:
predict_num

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,0,0,0,1,2,1,2020,4
1,0,0,0,1,2,1,2020,7
2,0,0,0,1,2,2,2020,4
3,0,0,0,1,2,2,2020,7
4,0,0,0,2,1,2,2020,4
...,...,...,...,...,...,...,...,...
143554,16,40,16,5,1,5,2020,7
143555,16,40,16,5,2,5,2020,4
143556,16,40,16,5,2,5,2020,7
143557,16,40,16,6,1,5,2020,4


In [85]:
# 예측
pred = model.predict(predict_num)
pred = np.expm1(pred)
predict_num['AMT'] = np.round(pred, 0)
predict_num['REG_YYMM'] = predict_num['year']*100 + predict_num['month']
predict_num = predict_num[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
predict_num = predict_num.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)


In [86]:
predict_num['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(predict_num['CARD_SIDO_NM'])
predict_num['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(predict_num['STD_CLSS_NM'])


In [89]:
# 제출 파일 만들기
submission = pd.read_csv('submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(predict_num, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission = submission.fillna(0)
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,9.174562e+07
1,202004,강원,골프장 운영업,3.464800e+09
2,202004,강원,과실 및 채소 소매업,8.996879e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,1.438203e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.130425e+09
1390,202007,충북,한식 음식점업,1.485431e+10
1391,202007,충북,호텔업,1.909843e+07
